This File is here to show how my machine learning algorithm works

Preparing the data

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
data = pd.read_csv("DNN_data.csv", parse_dates = ["time"], index_col = "time")
symbol = data.columns[0]
data["returns"] = np.log(data[symbol] / data[symbol].shift())
data

Adding lables/features to the data

In [ ]:
window = 50
df = data.copy()
#Creates the direction feature
df["dir"] = np.where(df["returns"] > 0, 1, 0)
#Created a feature based on a simple moving average
df["sma"] = df[symbol].rolling(window).mean() - df[symbol].rolling(150).mean()
#Creates a feature based on bollinger bands
df["boll"] = (df[symbol] - df[symbol].rolling(window).mean()) / df[symbol].rolling(window).std()
#Creates a feature based on the min price in a 20min time print
df["min"] = df[symbol].rolling(window).min() / df[symbol] - 1
#Creates a feature based in the max price in a 20min time print
df["max"] = df[symbol].rolling(window).max() / df[symbol] - 1
#Creates a feature based on momentum
df["mom"] = df["returns"].rolling(3).mean()
#Created a feature based on the trading volume in a given 20min time print
df["vol"] = df["returns"].rolling(window).std()
df.dropna(inplace = True)
df

Adding feature lags to the dataset

In [ ]:
lags = 5
cols = []
features = ["dir", "sma", "boll", "min", "max", "mom", "vol"]
for f in features:
        for lag in range(1, lags + 1):
            col = "{}_lag_{}".format(f, lag)
            df[col] = df[f].shift(lag)
            cols.append(col)
df.dropna(inplace = True)

Splitting the data into a train and a test set

In [ ]:
split = int(len(df)*0.66)
split

In [ ]:
train = df.iloc[:split].copy()
train

In [ ]:
test = df.iloc[split:].copy()
test

Feature standardization

In [ ]:
# train set parameters (mu, std) for standardization
mu, std = train.mean(), train.std()
# standardization of train set features
train_s = (train - mu) / std 

Creating the DND model

In [ ]:
from DNNModel import *
# fitting a DNN model with 3 Hidden Layers (50 nodes each) and dropout regularization

set_seeds(100)
model = create_model(hl = 3, hu = 50, dropout = True, input_dim = len(cols))
model.fit(x = train_s[cols], y = train["dir"], epochs = 50, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(train))

model.evaluate(train_s[cols], train["dir"]) # evaluate the fit on the train set
pred = model.predict(train_s[cols]) # prediction (probabilities)
pred

In [ ]:
#Shows the distribution of predictions#
plt.hist(pred, bins = 50)
plt.show()

Out-sample prediction and forward testing

In [ ]:
test_s = (test - mu) / std # standardization of test set features (with train set parameters!!!)
model.evaluate(test_s[cols], test["dir"])
pred = model.predict(test_s[cols])

In [ ]:
#Plotting a histogram of prediction values
plt.hist(pred, bins = 50);

In [ ]:
#Setting the condition in which to place a trade
test["proba"] = model.predict(test_s[cols])
test["position"] = np.where(test.proba < 0.45, -1, np.nan) # 1. short where proba < 0.47
test["position"] = np.where(test.proba > 0.55, 1, test.position) # 2. long where proba > 0.53
test.index = test.index.tz_localize("UTC")
test["NYTime"] = test.index.tz_convert("America/New_York")
test["hour"] = test.NYTime.dt.hour
test["position"] = np.where(~test.hour.between(2, 12), 0, test.position) # 3. neutral in non-busy hours
test["position"] = test.position.ffill().fillna(0) # 4. in all other cases: hold position
test.position.value_counts(dropna = False)
test["strategy"] = test["position"] * test["returns"]
test["creturns"] = test["returns"].cumsum().apply(np.exp)
test["cstrategy"] = test["strategy"].cumsum().apply(np.exp)
test[["creturns", "cstrategy"]].plot(figsize = (12, 8))
#Plot the trading results before trading costs are added
plt.show()

In [ ]:
#Testing with trading costs
ptc = 0.00004
test["trades"] = test.position.diff().abs()
#Display the number of trades taken
test.trades.value_counts()

In [ ]:
#Plotting the results
test["strategy_net"] = test.strategy - test.trades * ptc
test["cstrategy_net"] = test["strategy_net"].cumsum().apply(np.exp)
test[["creturns", "cstrategy", "cstrategy_net"]].plot(figsize = (12, 8))
plt.show()

Code to save the model and the parameters

In [34]:
model.save("DNN_model")
import pickle
params = {"mu":mu, "std":std}
pickle.dump(params, open("params.pkl", "wb"))

2022-03-24 19:48:27.245413: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: DNN_model/assets


Connecting the trading bot to the broker and setting the trading conditions

In [35]:
import pandas as pd
import numpy as np
import tpqoa
import fxcmpy
from datetime import datetime, timedelta
import time

In [36]:
# Loading the model
import keras
model = keras.models.load_model("DNN_model")
# Loading mu and std
import pickle
params = pickle.load(open("params.pkl", "rb"))
mu = params["mu"]
std = params["std"]

In [37]:
class DNNTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, lags, model, mu, std, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        self.lags = lags
        self.model = model
        self.mu = mu
        self.std = std
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M").c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            print (pd.to_datetime(datetime.utcnow()).tz_localize("UTC"))
            print (self.last_bar)
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                self.start_time = pd.to_datetime(datetime.utcnow()).tz_localize("UTC") # NEW -> Start Time of Trading Session
                break
                
    def on_success(self, time, bid, ask):
        print(self.ticks, end = " ", flush = True)
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        #create features
        df = df.append(self.tick_data) # append latest tick (== open price of current bar)
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["dir"] = np.where(df["returns"] > 0, 1, 0)
        df["sma"] = df[self.instrument].rolling(self.window).mean() - df[self.instrument].rolling(150).mean()
        df["boll"] = (df[self.instrument] - df[self.instrument].rolling(self.window).mean()) / df[self.instrument].rolling(self.window).std()
        df["min"] = df[self.instrument].rolling(self.window).min() / df[self.instrument] - 1
        df["max"] = df[self.instrument].rolling(self.window).max() / df[self.instrument] - 1
        df["mom"] = df["returns"].rolling(3).mean()
        df["vol"] = df["returns"].rolling(self.window).std()
        df.dropna(inplace = True)
        
        # create lags
        self.cols = []
        features = ["dir", "sma", "boll", "min", "max", "mom", "vol"]

        for f in features:
            for lag in range(1, self.lags + 1):
                col = "{}_lag_{}".format(f, lag)
                df[col] = df[f].shift(lag)
                self.cols.append(col)
        df.dropna(inplace = True)
        
        # standardization
        df_s = (df - self.mu) / self.std
        # predict
        df["proba"] = self.model.predict(df_s[self.cols])
        
        #determine positions
        df = df.loc[self.start_time:].copy() # starting with first live_stream bar (removing historical bars)
        df["position"] = np.where(df.proba < 0.47, -1, np.nan)
        df["position"] = np.where(df.proba > 0.53, 1, df.position)
        df["position"] = df.position.ffill().fillna(0) # start with neutral position if no strong signal
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  

In [38]:
trader = DNNTrader("oanda.cfg", "EUR_USD", bar_length = "20min",
                   window = 50, lags = 5, model = model, mu = mu, std = std, units = 10)

In [39]:
trader.get_most_recent()
trader.stream_data(trader.instrument, stop = 10000)
if trader.position != 0:
    close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units,
                                      suppress = True, ret = True) 
    trader.report_trade(close_order, "GOING NEUTRAL")
    trader.position = 0

2022-03-24 19:49:08.122958+00:00
2022-03-24 19:40:00


TypeError: Timestamp subtraction must have the same timezones or no timezones

In [ ]:
trader.data